#  Educational courses

In [1]:
import pandahouse as ph

In [2]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database': 'default',
                      'user': 'student', 
                      'password': 'dpo_python_2020'}

### 1. Очень усердные ученики.
#### Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

#### Задание
Дана таблица default.peas:

| Название атрибута | Тип атрибута | Смысловое значение                       |
|-------------------|--------------|------------------------------------------|
| st_id             | int          | ID ученика                               |
| timest            | timestamp    | Время решения карточки                   |
| correct           | bool         | Правильно ли решена горошина?            |
| subject           | text         | Дисциплина, в которой находится горошина |


Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов. NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [3]:
q = '''
    SELECT COUNT(DISTINCT st_id)
    FROM (SELECT st_id 
        FROM default.peas
        GROUP BY st_id
        HAVING SUM(correct) >= 20)
    '''

q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,uniqExact(st_id)
0,136


### 2. Оптимизация воронки

#### Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

#### Задача
Дана таблицы: default.peas (см. выше), default.studs:

| Название атрибута | Тип атрибута | Смысловое значение                   |
|-------------------|--------------|--------------------------------------|
| st_id             | int          |  ID ученика                          |
| test_grp          | text         |  Метка ученика в данном эксперименте |

и default.final_project_check:

| Название атрибута | Тип атрибута | Смысловое значение                     |
|-------------------|--------------|----------------------------------------|
| st_id             | int          | ID ученика                             |
| sale_time         | timestamp    | Время покупки                          |
| money             | int          | Цена, по которой приобрели данный курс |
| subject           | text         |                                        |

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
- ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.


In [4]:
q = '''
    SELECT test_grp,
        -- ARPU
        SUM(d.total_money) / COUNT(a.st_id) AS ARPU,
        
        -- ARPAU
        SUM(IF(b.active, d.total_money, 0)) / SUM(b.active) AS ARPAU,
        
        -- CR user to purchase
        SUM(d.total_money > 0) / COUNT(a.st_id) AS CR1,
        
        -- CR active user to purchase
        SUM(d.total_money > 0 AND b.active = 1) / SUM(b.active) AS CR2,
        
        -- CR math active user to math course purchase
        SUM(d.math_money > 0 AND c.active_math = 1) / SUM(c.active_math) AS CR3 

    -- data with unique st_id and allocated group
    FROM default.studs a

    -- join active users flag (> 10 correct answers in total)
    LEFT JOIN 
            (
            SELECT st_id,
            CASE
                WHEN st_id IN (SELECT a.st_id 
                                FROM default.studs a
                                LEFT JOIN default.peas b ON a.st_id = b.st_id
                                GROUP BY a.st_id
                                HAVING SUM(b.correct) > 10) THEN 1
                ELSE 0
            END AS active
            FROM default.studs
            ) b ON a.st_id = b.st_id

    -- join math active users flag (>= 2 correct answers in math)
    LEFT JOIN 
            (
            SELECT st_id,
            CASE
                WHEN st_id IN (SELECT DISTINCT a.st_id 
                    FROM default.studs a
                    LEFT JOIN default.peas b ON a.st_id = b.st_id AND b.subject = 'Math'
                    GROUP BY a.st_id
                    HAVING SUM(b.correct) >= 2) THEN 1
                ELSE 0
            END AS active_math
            FROM default.studs
            ) c ON a.st_id = c.st_id

    -- join payments data
    LEFT JOIN 
            (
            SELECT st_id, SUM(money) AS total_money, -- total payments sum per subject
                    SUM(CASE WHEN subject = 'Math' THEN money ELSE 0 END) AS math_money -- math payments sum per subject
            FROM default.final_project_check
            GROUP BY st_id
            ) d ON a.st_id = d.st_id
    GROUP BY test_grp
    '''

q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,test_grp,ARPU,ARPAU,CR1,CR2,CR3
0,control,4540.983607,10393.700787,0.049180,0.110236,0.061224
1,pilot,11508.474576,29739.583333,0.108475,0.260417,0.095238
